In [1]:
import pandas as pd

In [2]:
Path = r'E:\_Projects\NFC\\'
NFC_Path = r'E:\_Projects\NFC\Input\NFC_Files\\'

In [3]:
print('Automation Started')

Automation Started


In [4]:
####### 6M Tracker 
Tracker_6M = pd.read_excel(Path+'Input\\NFC Tracker.xlsx',sheet_name='6M NFC')
Tracker_6M = Tracker_6M[Tracker_6M['Config Status']=='Completed']

######### 1Year Tracker

Tracker_6M['Country'] = Tracker_6M['Country'].fillna('NA')

Tracker_6M.to_csv(Path+'Output//ref//Tracker_6M.csv',index=False)


e:\softwares\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [5]:

######### 1Year Tracker

Tracker_1Y = pd.read_excel(Path+'Input\\NFC Tracker.xlsx',sheet_name='1 Year')

Tracker_1Y = Tracker_1Y[Tracker_1Y['Config Status']=='Completed']
Tracker_1Y['Country'] = Tracker_1Y['Country'].fillna('NA')
Tracker_1Y['Class'] = Tracker_1Y['Class'].astype(str)


########  Load New Demand
New_Demand = pd.read_csv(Path+'Input\\New Demand.csv')

New_Demand_Bk = New_Demand

New_Demand = New_Demand[New_Demand['Total']>1]

New_Demand['Class'] = New_Demand['Class'].astype(str)
New_Demand = New_Demand[['Class','Country Code', 'Key Account','Total','Partial config']]
New_Demand['Country Code'] = New_Demand['Country Code'].fillna('NA')

New_Demand.rename(columns={'Country Code' : 'Country'}, inplace=True)


New_Demand = New_Demand.pivot_table(
    index=['Class','Country', 'Key Account','Partial config'],
    values='Total',
    aggfunc='sum'  # You can change this to other aggregation functions like 'mean', 'max', etc.
)
New_Demand.to_csv(Path+'Output//ref//New Demand.csv')

C:\Users\vv185114\AppData\Local\Temp\2\ipykernel_14072\3328959728.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Demand['Class'] = New_Demand['Class'].astype(str)


In [6]:
New_Demand = pd.read_csv(Path+'Output//ref//New Demand.csv')

#print(New_Demand.columns)
#print(Tracker_1Y.columns)

New_Demand['Class'] = New_Demand['Class'].astype(str)

Tracker_1Y = pd.merge(Tracker_1Y, New_Demand, on=['Class','Country' , 'Key Account'], how='left')

Tracker_1Y = Tracker_1Y[Tracker_1Y['Partial config'] == 'Yes']
#print(Tracker_1Y.columns)

Tracker_1Y.to_csv(Path+'Output//ref//Tracker_1Y.csv',index=False)

# List of columns to delete
columns_to_delete = ['Partial config','Total']

# Drop the specified columns
Tracker_1Y.drop(columns=columns_to_delete, inplace=True)

#Tracker_1Y.to_csv(Path+'Output//ref//Tracker_1Y.csv',index=False)


In [7]:
Tracker = pd.concat([Tracker_6M,Tracker_1Y], ignore_index=True)

C:\Users\vv185114\AppData\Local\Temp\2\ipykernel_14072\2129137194.py:1: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  Tracker = pd.concat([Tracker_6M,Tracker_1Y], ignore_index=True)


In [8]:
# Convert 'Date Submitted' to datetime
Tracker['Date Submitted'] = pd.to_datetime(Tracker['Date Submitted'], format='%d-%b-%Y')

# Create new columns for year, month, and day with two-digit day
Tracker['Year_Split'] = (Tracker['Date Submitted'].dt.year).astype(str)
Tracker['Month_Split'] = (Tracker['Date Submitted'].dt.strftime('%b')).astype(str)
Tracker['Month_Split_No'] = (Tracker['Date Submitted'].dt.strftime('%m')).astype(str)
Tracker['Date_Split'] = (Tracker['Date Submitted'].dt.strftime('%d')).astype(str)

# Convert 'date' column to datetime and format it
Tracker['Date Submitted'] = pd.to_datetime(Tracker['Date Submitted']).dt.strftime('%d-%b-%Y')

Tracker.to_csv(Path+'Output//ref//NFC Tracker.csv',index=False)


In [9]:
# Check for duplicates in the specified columns
duplicates = Tracker[Tracker.duplicated(subset=['Key Account', 'Master Customer Name', 'Class', 'Country'], keep=False)]
duplicates = duplicates[['Key Account', 'Master Customer Name', 'Class', 'Country']]
duplicates.drop_duplicates(inplace=True)

print('#####################################################################\n')
if not duplicates.empty:
    print("            Duplicate config found in Tracker:")

    print('\n#####################################################################\n')
    print(duplicates)

    print('\n#####################################################################\n')
else:
    print("                         No duplication in Tracker")
    print('#####################################################################\n')

#####################################################################

                         No duplication in Tracker
#####################################################################



In [10]:
User_input = input('Press Enter to Continue : ')

Press Enter to Continue : 


In [11]:
try:
    Tracker['Class'] = Tracker['Class'].astype(float).astype(int)
    Tracker['Class'] = Tracker['Class'].astype(str)
except:
    Tracker['Class'] = Tracker['Class'].astype(str)
    
Tracker['Location'] = NFC_Path + Tracker['Year_Split'] +'\\' +Tracker['Month_Split_No'] + "-" + Tracker['Month_Split'] +"\\" +"Output" + "\\" +Tracker['Date_Split']  + "\\" + Tracker['Class']+ " "+Tracker['Key Account'] + ".xlsx"

#print(Tracker['Location'])

Tracker.to_csv(Path+'Output\\Tracker.csv',index=False)


In [12]:
import os

file_not_found = pd.DataFrame(columns=['Location'])

# Initialize an empty dataframe to store consolidated data
consolidated_df = pd.DataFrame()

# Iterate over each row in the nfc_df
for index, row in Tracker.iterrows():
    location = row['Location']
    
    # Check if the file exists
    if os.path.exists(location):
        print('Config Pulled from Location : ', location)
        # Try to read the 'Mix' sheet
        try:
            temp_df = pd.read_excel(location, sheet_name='Mix')
        except ValueError:
            # If 'Mix' sheet is not available, read the 'F&D' sheet
            temp_df = pd.read_excel(location, sheet_name='F&D')
        
        # Add the additional columns from nfc_df to temp_df
        temp_df['Key Account'] = row['Key Account']
        temp_df['Class'] = row['Class']
        temp_df['Master Customer Name'] = row['Master Customer Name']
        temp_df['Country'] = row['Country']
        temp_df['Date Submitted'] = row['Date Submitted']
        
        # Append the data to the consolidated dataframe
        consolidated_df = pd.concat([consolidated_df, temp_df], ignore_index=True)
    else:
        #continue
        file_not_found = pd.concat([file_not_found, pd.DataFrame({'Location': [location]})], ignore_index=True)
        
try:
    consolidated_df = consolidated_df[['Date Submitted','Class','Key Account','Master Customer Name','Country','Item','Quantity','Mix']]
except:
    consolidated_df = consolidated_df[['Date Submitted','Class','Key Account','Master Customer Name','Country','Item','Quantity']]
   
# Store the consolidated dataframe to the output location
output_location = Path+'Output//ref//consolidated_file.xlsx'
consolidated_df.to_excel(output_location, index=False)

print(f"Consolidated data has been stored to {output_location}.")

Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\20\6627 OTHER-FI.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\20\6681 CIBC.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\20\6659 OTHER-BO.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\21\6627 OTHER-NG.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\26\6623 OTHER-QA.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\26\2062 OTHER-QA.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\26\6628 OTHER-QA.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\26\6682 OTHER-QA.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\08-Aug\Output\26\6684 OTHER-QA.xlsx
Config Pulled from Location :  E:\_Projects\NFC\In

Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\26\6627 OTHER-ES.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\26\6684 BANCO DE SABADELL.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\26\6627 BBVA.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\27\6623 OTHER-FI.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\27\2062 PRINTEC.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\27\2064 PRINTEC.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\27\2012 FCTI INC.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\27\6628 OTHER-EG.xlsx
Config Pulled from Location :  E:\_Projects\NFC\Input\NFC_Files\\2024\11-Nov\Output\27\2018 HITACHI PAYMENT SERVICES PVT.xlsx
Config Pulled from Loca

In [13]:
####### Config Not found

# Check if there are any rows in the DataFrame
if not file_not_found.empty:
    print('#################################################\n')
    print("         Following Config's are not found : ")
    print('\n#################################################')
    for x in file_not_found['Location']:
        print(x)
else:
    print('#################################################\n')
    print('                   All config Found')
    print('#################################################\n')


#################################################

                   All config Found
#################################################



In [14]:
print('#####################################################################\n')
User_input = input('               Press Enter to Continue : ')
print('\n#####################################################################\n')

#####################################################################

               Press Enter to Continue : 

#####################################################################



In [15]:
####### Consolidate file data workout
consolidated_df.loc[consolidated_df['Quantity'].notnull(),'NFC Split AR %'] = consolidated_df['Quantity']

try:
    consolidated_df.loc[consolidated_df['Mix'].notnull(),'NFC Split AR %'] = consolidated_df['Mix']
except:
    pass
    
consolidated_df['Comp Qty'] = consolidated_df['NFC Split AR %']

try:
    consolidated_df.loc[consolidated_df['Mix'].notnull(),'Comp Qty'] = 1
except:
    pass
    
consolidated_df['AR %'] = consolidated_df['NFC Split AR %'] / consolidated_df['Comp Qty']


In [16]:
try:
    # List of columns to delete
    columns_to_delete = ['Quantity', 'Mix']
    consolidated_df.drop(columns=columns_to_delete, inplace=True)
except:
    # List of columns to delete
    columns_to_delete = ['Quantity']
    consolidated_df.drop(columns=columns_to_delete, inplace=True)


####### Rename

consolidated_df.rename(columns={'Date Submitted' : 'Date'}, inplace = True)
consolidated_df.rename(columns={'Master Customer Name' : 'Master Customer'}, inplace = True)
consolidated_df.rename(columns={'Country' : 'Country Code'}, inplace = True)

# Store the consolidated dataframe to the output location
#consolidated_df.to_csv(Path+'Output//Consolidate.csv', index=False)


In [17]:
########### Loading NFC File

NFC_Config = pd.read_excel(Path+'Input//NFC Configuration.xlsx',sheet_name='NFC Configuration')

# Convert 'date' column to datetime and format it
NFC_Config['Date'] = pd.to_datetime(NFC_Config['Date']).dt.strftime('%d-%b-%Y')

#print(NFC_Config)


In [18]:
NFC_Config['Class'] = NFC_Config['Class'].astype(str)
Tracker['Class'] = Tracker['Class'].astype(str)

Tracker_Validation = Tracker[['Date Submitted','Class','Country' , 'Key Account']]

NFC_Config = pd.merge(NFC_Config, Tracker_Validation, left_on = ['Date','Class', 'Country Code', 'Key Account'], right_on = ['Date Submitted','Class','Country' , 'Key Account'], how='left')

#NFC_Config.to_csv(Path+'Output//NFC_Validation.csv',index=False)

In [19]:
# Keep only the 'Date Submitted' column and drop rows with blank values
NFC_Config = NFC_Config[NFC_Config['Date Submitted'].replace('', pd.NA).notna()]


# List of columns to delete
columns_to_delete = ['CC - Cls+Ctry+KA', 'Category' ,'Item Description','Publish','Comments','PDN','Replacement','Current Discont','Plant','Missing','New demand','OTHER','NFC Tracker','commends','Date Submitted', 'Country']

# Drop the specified columns
NFC_Config.drop(columns=columns_to_delete, inplace=True)

#print(NFC_Config.columns)

#NFC_Config.to_csv(Path+'Output//NFC_Config_Filter.csv',index=False)

In [20]:
NFC_Config['Class'] = NFC_Config['Class'].astype(str)

NFC_Config_Validation = NFC_Config[['Date','Class', 'Country Code', 'Key Account']]

NFC_Config_Validation.drop_duplicates(inplace=True)


NFC_Config_Validation['NFC_Validation'] = 'Yes'

#NFC_Config_Validation.to_csv(Path+'Output//NFC_Config_Validation.csv',index=False)

consolidated_df['Class'] = consolidated_df['Class'].astype(str)

consolidated_df = pd.merge(consolidated_df, NFC_Config_Validation, on = ['Date','Class', 'Country Code', 'Key Account'], how='left')

#consolidated_df.to_csv(Path+'Output//Consolidate_1.csv', index=False)

consolidated_df = consolidated_df[consolidated_df['NFC_Validation'].isnull()]

#consolidated_df.to_csv(Path+'Output//Consolidate_2.csv', index=False)


In [21]:
##### Combain 2 files to single file

NFC_Config = pd.concat([NFC_Config,consolidated_df], ignore_index=True)

NFC_Config.to_csv(Path+'Output//ref//NFC Configuration.csv',index=False)

In [22]:
##### Grouping & Discont

In [23]:
GD = pd.read_excel(Path+'Input//Groupings & Discontinuation.xlsb',sheet_name='Groupings', engine='pyxlsb')

In [24]:
GD.drop_duplicates(subset=['Item'],keep='first' ,inplace=True)

GD.rename(columns={'PDN Remarks':'PDN'}, inplace=True)
GD.rename(columns={'Replacement Feature':'Replacement'}, inplace=True)
GD.rename(columns={'Current Discontinuation':'Current Discont'}, inplace=True)
GD.rename(columns={'Description':'Item Description'}, inplace=True)

NFC_Config = pd.merge(NFC_Config, GD[['Item','PDN', 'Replacement', 'Current Discont','Item Description']], on=['Item'], how='left')

#print(NFC_Config.columns)

In [25]:
###############  New Demand

In [26]:

New_Demand = New_Demand_Bk

New_Demand['Class'] = New_Demand['Class'].astype(str)
New_Demand = New_Demand[['Class','Country Code', 'Key Account','Plant','Partial config']]
New_Demand['Country Code'] = New_Demand['Country Code'].fillna('NA')
#New_Demand.rename(columns={'Country Code' : 'Country'}, inplace=True)
New_Demand.rename(columns={'Partial config' : 'Missing'}, inplace=True)

New_Demand.drop_duplicates(subset=['Class','Country Code', 'Key Account'],keep='first', inplace=True)


NFC_Config = pd.merge(NFC_Config, New_Demand, on=['Class','Country Code', 'Key Account'], how='left')



C:\Users\vv185114\AppData\Local\Temp\2\ipykernel_14072\2693154469.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Demand['Country Code'] = New_Demand['Country Code'].fillna('NA')
C:\Users\vv185114\AppData\Local\Temp\2\ipykernel_14072\2693154469.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Demand.rename(columns={'Partial config' : 'Missing'}, inplace=True)
C:\Users\vv185114\AppData\Local\Temp\2\ipykernel_14072\2693154469.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

In [27]:
New_Demand = New_Demand_Bk
#New_Demand.rename(columns={'Country Code' : 'Country'}, inplace=True)

New_Demand = New_Demand.pivot_table(
    index=['Class','Country Code', 'Key Account'],
    values='9M',
    aggfunc='sum'  # You can change this to other aggregation functions like 'mean', 'max', etc.
)
New_Demand.to_csv(Path+'Output//ref//New Demand.csv')

In [28]:
New_Demand = pd.read_csv(Path+'Output//ref//New Demand.csv')


New_Demand['Class'] = New_Demand['Class'].astype(str)
New_Demand.rename(columns={'9M' : 'New demand'}, inplace=True)


NFC_Config = pd.merge(NFC_Config, New_Demand, on=['Class','Country Code', 'Key Account'], how='left')



In [29]:
######### Multipl customer Validation

In [30]:

Multi_Cust = pd.read_csv(Path+'Input//Multiple_Customer.csv')

Multi_Cust.drop_duplicates(inplace=True)


NFC_Config['Key_Account_Validation'] = NFC_Config['Key Account'].str.upper()
Multi_Cust['Key_Account_Validation'] = Multi_Cust['Key_Account_Validation'].str.upper()

NFC_Config = pd.merge(NFC_Config, Multi_Cust, on=['Key_Account_Validation'] , how='left')

NFC_Config.loc[NFC_Config['Key_Account_Validation'].str.contains('OTHER-'),'OTHER'] = 'Yes'
NFC_Config['OTHER'] = NFC_Config['OTHER'].fillna('No')


In [31]:
########### Publish Flag

In [32]:

import glob

files = glob.glob(Path+'\\Input\\ATLEOS OCP IND POR_???????????.csv')

#print(files)

POR = pd.DataFrame()


for x in files:
    #print(x)
    df = pd.read_csv(x,low_memory=False)
    POR = pd.concat([POR,df],axis=0)



In [33]:
POR = POR[POR['Data Series']=='NEW DEMAND']

Str_index = POR.columns.get_loc('Data Series')+1
End_index= POR.columns.get_loc('Comments')-1

#print(POR.columns[End_index])
#print(POR.columns[Str_index])

POR['Total'] = POR.iloc[:, Str_index:End_index + 1].sum(axis=1)

POR = POR[POR['Total']>=1]

POR = POR[['Class','Key Account','Master Customer','Country Code','Total']]

### Create PIvot

POR = POR.pivot_table(
    index=['Class','Key Account','Master Customer','Country Code'],
    values='Total',
    aggfunc='sum' 
)


POR.to_csv(Path+'Output//ref//IND_POR.csv')


In [34]:
#### COunt Customers

POR = pd.read_csv(Path+'Output//ref//IND_POR.csv')

POR.rename(columns={'Master Customer' : 'Customer_Count'},inplace=True)

POR = POR.pivot_table(
    index=['Class','Key Account','Country Code'],
    values='Customer_Count',
    aggfunc='count' 
)

POR.to_csv(Path+'Output//ref//POR_Customer.csv')

In [35]:
POR = pd.read_csv(Path+'Output//ref//IND_POR.csv')

POR = POR[['Class', 'Key Account', 'Master Customer', 'Country Code']]

POR.drop_duplicates(inplace=True)

POR_Customer_Count = pd.read_csv(Path+'Output//ref//POR_Customer.csv')

POR = pd.merge(POR, POR_Customer_Count, on=['Class','Country Code', 'Key Account'], how='left')

POR = POR[POR['Customer_Count']==1]

POR.rename(columns={'Master Customer' : 'Single_Customer'},inplace=True)

POR = POR[['Class','Country Code', 'Key Account','Single_Customer']]

POR.to_csv(Path+'Output//ref//Single_Customer.csv',index=False)


In [36]:
#### Load Customers
POR_Customer = pd.read_csv(Path+'Output//ref//POR_Customer.csv')

POR_Customer['Class'] = POR_Customer['Class'].astype(str)

Single_Customer = pd.read_csv(Path+'Output//ref//Single_Customer.csv')

Single_Customer['Class'] = Single_Customer['Class'].astype(str)
###############
NFC_Config = pd.merge(NFC_Config, POR_Customer, on=['Class','Country Code', 'Key Account'], how='left')

NFC_Config = pd.merge(NFC_Config, Single_Customer, on=['Class','Country Code', 'Key Account'], how='left')

#NFC_Config.to_csv(Path+'Output//NFC_Config_Customer_Details.csv',index=False)

In [39]:
NFC_Config.loc[(NFC_Config['OTHER'] == 'Yes') & (NFC_Config['Customer_Count'] > 1), 'Publish'] = 'No'
NFC_Config.loc[(NFC_Config['OTHER'] == 'Yes') & (NFC_Config['Customer_Count'] == 1) & (NFC_Config['Master Customer'] != NFC_Config['Single_Customer']), 'Publish'] = 'No'

NFC_Config['Publish'] = NFC_Config['Publish'].fillna('Yes')
     
NFC_Config['New demand'] = NFC_Config['New demand'].fillna(0)
NFC_Config.loc[NFC_Config['New demand']<=0, 'Publish'] = 'Yes'

#NFC_Config.to_csv(Path+'Output//ref//NFC Configuration.csv',index=False)

In [ ]:
Tracker = pd.read_csv(Path+'Output//ref//NFC Tracker.csv')
Tracker['Class'] = Tracker['Class'].astype(str)

Tracker = Tracker[['Key Account','Master Customer Name','Class','Country','Date Submitted','Config Status']]

########## Date Formate
Tracker['Date Submitted'] = pd.to_datetime(Tracker['Date Submitted']).dt.strftime('%d-%b-%Y')
Tracker['NFC Tracker'] = Tracker['Date Submitted']

NFC_Config['Date'] = pd.to_datetime(NFC_Config['Date']).dt.strftime('%d-%b-%Y')


Tracker.rename(columns={'Date Submitted' : 'Date'}, inplace=True)
Tracker.rename(columns={'Config Status' : 'Comments'}, inplace=True)
Tracker.rename(columns={'Master Customer Name' : 'Master Customer'}, inplace=True)
Tracker.rename(columns={'Country' : 'Country Code'}, inplace=True)


########## Clubbing
NFC_Config = pd.merge(NFC_Config, Tracker, on=['Date','Key Account','Master Customer','Class','Country Code'], how='left')

########## Create Columns
NFC_Config['Category'] = 'NFC'

########## Concat
NFC_Config['CC - Cls+Ctry+KA'] = NFC_Config['Class'] + NFC_Config['Country Code'] + NFC_Config['Key Account']

NFC_Config = NFC_Config[['CC - Cls+Ctry+KA','Category','Date','Class','Key Account','Master Customer','Country Code','Item','Item Description','NFC Split AR %','Comp Qty','AR %','Publish','Comments','PDN','Replacement','Current Discont','Plant','Missing','New demand','OTHER','NFC Tracker','Comments','RFC','RFC Qty','Diff','Single_Customer']]

NFC_Config.to_csv(Path+'Output//NFC Configuration.csv',index=False)

In [ ]:
print('#####################################################################\n')
print('                   Automation Completed Successfully')
print('\n#####################################################################\n')